## Simple Linear Regression

<div class="alert alert-block alert-info"><b>Instructor Note:</b> This notebook is intended to introduce students to simple linear regression. We'll work with a dataset that contains information about various types of fish. The <code>fish</code> dataset was submitted by Juha Puranen from the University of Helsinki and can be found <a href = "https://jse.amstat.org/datasets/fishcatch.dat.txt" >here</a>.
<br>In this notebook, you will...
<li>Fit a simple regression models </li><li>Interpret the coefficients of simple regression models </li><li>Compare the strength of different simple regression models</li></div>

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command downloads and installs a useful package of R
# commands for constructing visualizations
install.packages("ggformula", quiet = TRUE, verbose = FALSE)

# This command loads useful packages of R commands into your
# notebook for manipulating data and constructing visualizations
suppressPackageStartupMessages({
    library(dplyr)
    library(ggplot2)
    library(ggformula)
})

# This command sets the theme for the visualizations in
# this notebook
theme_set(
  theme_light() +
  theme(
    panel.background = element_rect(fill = "white", colour = "white"),
    plot.background = element_blank(),
    axis.line = element_line(color = "black")
  )
)

#### The Dataset (`fish.csv`)
In this notebook, we'll be using the `fish.csv` file, which includes data about 7 common different fish species in fish market sales.

This dataset originally comes from Juha Puranen from the Departement of Statistics at the University of Helsinki, Finland. The researchers wanted to develop a model that could be used to predict the weight of a fish.


##### Variables
- `Species` --> categorical | Perch, Bream, Other
- `Weight` --> quantitative | Weight of the fish (in grams)
- `Length1` --> quantitative | Length from the nose to the beginning of the tail (in cm)
- `Length2` --> quantitative | Length from the nose to the notch of the tail (in cm)
- `Length3` --> quantitative | Length from the nose to the end of the tail (in cm)
- `Height` --> quantitative | Maximal height as % of Length3
- `Width` --> quantitative | Maximal width as % of Length3  

##### Detailed description, diagram of measurements and abstract notes can be found <a href="https://jse.amstat.org/datasets/fishcatch.txt">here</a>.

### 1.0 - Exploratory Data Analysis

To begin, let's download our data. We'll download the `fish.csv` file and store it in an R dataframe called `dat`.

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command saves the fish.csv to a dataframe
url <- 'https://raw.githubusercontent.com/mahmoudharding/mat2/main/lesson14/data/fish.csv'
dat <- read.csv(url)

**Question 1.1 -** Use the `head` command to print out the first several rows of the dataset.

In [ ]:
## Sample Response
# Print head of dataset
head(dat)

**Question 1.2 -** Use the `dim` command to find the number of fish (rows) and number of variables (columns) in our data set.

In [ ]:
## Sample Response
# Get dimensions of dataset
dim(dat)

A good measure to use to predict the weight of a fish could be to use it's total body length. When analyzing variables of interest (`Weight`, `Length3`), it's often helpful to calculate summary statistics. For quantitative variables, we can use the summary command to find the five-number summary (minimum, Q1, median, Q3, maximum) and the average (mean) of the values. The code block shows how we find these summary statistics for the `Weight` variable.

**Note:** The `$` sign in R is used to isolate a single variable (`Weight`) from a dataframe (`dat`).

In [ ]:
## Run this code but do not edit it
# Find summary statistics for Weight
summary(dat$Weight)

**Question 1.3 -** Comment on what these summary statistics reveal about the `Weight` values in our dataset.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> <li>The minimum weight is 32 grams.</li><li>Q3 is 687.5, meaning 75% of the fish weighed 650 grams or less.</li><li>The maximum weight is 1650 grams.</li><li>The mean is about 50% larger than the median which indicates that the distribution of weights is right-skewed.</li>
</div>

Now let's examine `Length3`. First we'll look at the summary statstics.

**Question 1.4 -** Use the `summary` command to find the 5 number summary for the variable `Length3`.

In [ ]:
## Sample Response
# Find summary statistics for Length3
summary(dat$Length3)

**Question 1.5 -** Comment on what these summary statistics reveal about the `Length3` values in our dataset.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> <li>The minimum length is 14.7 cm and the maximum length is 65.80 cm. This could mean that there are significant differences in the lengths for different species of fish.</li><li>Q3 is 40.55, meaning 75% of the fish measured were 40.55 cm or less.</li><li>The mean and the median are relatively close. This could mean that the full body lengths are uniformally distributed.</li></div>

It is also helpful to visualize the distribution of a quantitative variable. This can be useful in determining if the values are normally distributed or skewed (right or left). First we'll take a look at the distribution of the outcome variable `Weight`.

In [ ]:
## Run this code but do not edit it
# Create histogram for Weight
gf_histogram(~Weight, data = dat, fill = 'turquoise', color = 'black')

As we suspected from the summary statistics, it appears that most of the fish weigh less than 650 grams and that the distribution is right-skewed. Now let's take a look at the distribution of the `Length3` variable.

In [ ]:
## Run this code but do not edit it
# Create histogram for Weight
gf_histogram(~Length3, data = dat, fill = 'turquoise', color = 'black')

**Question 1.6 -** Comment on what the histogram reveals about the `Length3` values in our dataset.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> <li>The histogram appears to have two peaks - meaning that the lengths have a bi-modal distribution.</li><li>There are a few fish that have full body lengths that are greater than 60 cm.</li><li>Most of the fish have lengths that are between 0 and 30 cm and from 35 to 50 cm.</li></div>

For categorical data, it doesn't make sense to find means and medians. Instead, it's helpful to look at value counts and proportions. We can use the `table` command to find the counts of the different values for `Species`:

In [ ]:
## Run this code but do not edit it
# Find counts of values for different species of fish
species_counts <- table(dat$Species)
species_counts

To get a better sense of scale, we can turn these raw counts into proportions by dividing them by the total:

In [ ]:
## Run this code but do not edit it
# Find the total by summing of all counts in table
total <- sum(species_counts)

# Find the proportions for species by dividing table by total
species_counts / total

**Question 1.7 -** Comment on the proportion of species types.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b><li>The Bream and Perch species make up the majority of fish types in the data set.</li><li>Whitefish and Smelt together only account for about 4% of the fish species in the data set.</li>
</div>

Finally, to get a sense of how the weights and lengths are distributed across the various species we can create a box plot.

In [ ]:
## Run this code but do not edit it
# Create boxplots for weight for the different species
gf_boxplot(Weight ~ Species, data = dat, fill = 'turquoise', color = 'black')

<div class="alert alert-block alert-info"><b>Instructor Note:</b> By convention, in R's boxplots, outliers are visualized as dots. The edges of the box represent the first and third quartiles, and the line within the box represents the median. If students are unfamiliar with boxplots, we recommend <a href = "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/constructing-a-box-and-whisker-plot">this video</a> from Khan Academy (quick refresher) or <a href = "https://skewthescript.org/1-5">this lesson</a> from Skew The Script (detailed lesson with real-world context).
</div>

In this case, we're using `Species` as the **predictor variable** and `Weight` as the **outcome variable**. In other words, we can use the species type of a fish (Bream, Perch, Smelt, etc.) to help predict its weight. That's because weights of different species of fish are not the same. So, knowing the species of a fish can help us better predict its weight.

**Note:** This predictor-outcome relationship is coded in R through the syntax `outcome ~ predictor`, as in `gf_boxplot(Weight ~ Species,...)`.

We see that weight for Prakki and Roach tend to be a lot lower than the weights of Bream, Perch, Pike and Whitefish. We also notice that a few of the Roach species are outliers. And, the minimum weight for the Parkki, Perch and Roach seem to be the same.

### 2.0 - Scatter Plots

So, our main **outcome variable** in this analysis will be `Weight`. We're going to use scatterplots to see how strongly different **predictor variables** correlate with weight. In particular, we're to explore how well each of the following variables predicts weight:

- `Length3` - Length from the nose to the end of the tail (in cm)
- `Height` - Maximal height as % of Length3
- `Width` - Maximal width as % of Length3  

To begin, let's create a **scatterplot** of `Weight` and the `Length3`. We can use the `gf_point` command to make the graph:

In [ ]:
## Run this code but do not edit it.
# Create scatterplot: Weight ~ Length3
gf_point(Weight ~ Length3, data = dat)

**Question 2.1 -** Using your scatterplot, describe the relationship between `Length3` and `Weight`. For instance, are these variables positively or negatively related? How can you tell? Explain.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> <li>Weight and full body length (Length3) are positively related. As the length increases the weight of the fish increases. This makes sense because longer fish are probably bigger and as a result, probably weigh more.</li><li>The scatter plot also appears to be curved.</li><liv>The scatter plaot appears to show multiple positive relationships. There are a cluster of points on the x-axis (Length3) that start close to 0 and go up to 50, and a different cluster of points that start at 35 and go up to 70.</li>
</div>

### 3.0 - Simple Linear Regression (one predictor)

**Question 3.1 -** Watch <a href="https://youtu.be/hvWgu4A0VA4">this video</a>, which provides an introduction to simple linear regression. Note: This video is adapted from other materials and covers data from a separate context. However, the video provides a good intro to the concepts and models we'll be using in this section of the project.

<div class="alert alert-block alert-info"><b>Instructor Note:</b> If your students want to cover the discussion question at the end of the video, you can download the handout key or slide deck from the <a href = "https://skewthescript.org/3-2">full Skew The Script lesson page</a>, which has useful teaching materials for covering the discussion question. Note: Covering the discussion question is not required for successful completion of this project.
</div>

Let's create a linear regression model relating `Length3` (x) and `Weight` (y). To visualize our model, we can graph the line modeled by our equation on top of the scatterplot relating `Length3` to `Weight`. We use the `gf_point` command to produce the scatterplot, the `gf_lm` command to graph our linear model, and a `%>%` symbol to put the elements together on the same graph:

In [ ]:
## Run this code but do not edit it
# Overlay linear model of default_rate ~ pct_PELL on top of scatterplot
gf_point(Weight ~ Length3, data = dat) %>% gf_lm(color = "red")

**Question 3.2 -** Would the slope value of the linear regression model for these data be positive or negative? How can you tell?

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> The slope will be positive. In our scatter plot, as the value of the <code>Length3</code> variable increases, the value of the <code>Weight</code> variable also tends to increase.
</div>

R can help us find the equation that models this linear regression line. We can model a linear trend between a predictor $(x)$ and outcome $(y)$ using this linear regression formula:

$$
\hat{y} = \theta_{0} + \theta_{1}x
$$
Where:
- $\hat{y}$ is the predicted $y-$value (predicted outcome value)
- $\theta_{0}$ is the $y-$intercept --> the predicted $y-$value (outcome value) when $x=0$ (the predictor's value is 0)
- $\theta_{1}$ is the slope --> the predicted change in $y$ (outcome) for a 1-unit increase in $x$ (predictor)
- $x$ is the $x-$value (predictor value)

To fit a linear regression model to a set of data in R, we use the `lm` command. `lm` standars for "linear model." Here, we use `lm` to find the linear regression model relating `Length3` $(x)$ and `Weight` $(y)$.

In [ ]:
## Run this code but do not edit it
# Create and display linear model: Weight ~ Length3
length3_model <- lm(Weight ~ Length3, data = dat)
length3_model

The output of the `lm` command is a bit clunky, but here's what it means:
- The `(Intercept)` value is the y-intercept ($\theta_{0}$)
- The `Length3` value is the coefficient for the predictor. In other words, it's the slope ($\theta_{1}$)

So, our regression equation can be written as:

$$
\hat{y} = -593.72 + 31.17x
$$

**Question 3.3 -** Identify the slope value and interpret what it means (in context).

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> Slope = 31.17. For every 1 cm increase in full body length, our model predicts a 31.17 gram increase in weight.
</div>

**Question 3.4 -** Use the `gf_point` and `gf_lm` commands to visualize a linear regression model for predicting `Width` (outcome) using `Length1` (predictor).

In [ ]:
## Sample response
# Graph showing model for Weight ~ Width
gf_point(Weight ~ Width, data = dat) %>% gf_lm(color = "red")

**Question 3.5 -** Use the `lm` command to find the linear regression model you visualized above. Store the model in an object called `width_model`.

In [ ]:
## Sample response
# Linear model for Weight ~ Width
width_model <- lm(Weight ~ Width, data = dat)
width_model

**Question 3.6 -** Indentify the slope value and interpret what it means (in context).

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> Slope = 221.6. For every 1 cm increase in width, our model predicts a 221.6 gram increase in the weight of the fish.
</div>

### 4.0 - Analyzing strength $(R^2)$

In addition to the direction of a relationship (positive or negative), we can also look at the **strength** of a relationship. The strength is a measure of the **quality of our model's predictions.** A key metric for analyzing the strength of a model is $R^2$. The following diagram (from <a href = "https://skewthescript.org/3-3-a">Skew The Script</a>) shows the $R^2$ values of various linear models:

<img src="https://skewthescript.org/s/r_squared.PNG">

In the "weak" correlations, we see that our predictions (the linear model) tend to be far away from the actual data values (the points). If we used a model with weak correlation to predict **new** data values, our predictions would have high error. If we used a model with strong correlation to predict **new** data values, our predictions would have low error.

$R^2$ takes values between 0 - 1 (alternatively: $0\% - 100\%$). The stronger the model, the closer $R^2$ gets to 1 (or $100\%$). The weaker the model, the closer $R^2$ gets to 0 (or $0\%$). An intuitive way to think about it: for the perfectly strong correlations, the model gives 100% perfect predictions. The models explain $100\%$ of the variation in the data, so $R^2 = 100\%$. As the correlations get weaker, they start leaving room for error, since the models capture less of the variation in the data. So, the $R^2$ value declines from $100\%$, approaching $0\%$ if there's no correlation (model adds no prediction power compared to naive guessing).

**Optional Resource:** If you'd like a more thorough explanation of the math behind $R^2$, check out <a href="https://youtu.be/bMccdk8EdGo">this video</a>.

To see the $R^2$ values of our linear regression models, we can use the `summary` command. For example, here we get the `summary` printout of `length3_model` - the modeled we stored that relates `Weight` to `Length3`.

In [ ]:
## Run this code but do not edit it
# Summarize default_rate ~ grad_rate model
summary(length3_model)

There's a lot going on in this printout. For now, focus at the bottom of the printed information. The `Multiple R-squared` value is the $R^2$ value for the model. In this case, $R^2 = 85.68\%$. So, we can say that the correlation between full body length and weight is relatively strong. This model would yield relatively strong predictions for weight if used to predict on new full body fish lengths.

**Question 4.1 -** Let's consider a new variable: `Height`. How well does the height of a fish predict its weight? Let's start exploring. Go ahead and create a scatterplot that visualizes the relationship between `Height` (predictor) and `Weight` (outcome). Overlay a linear regression model on the graph.

In [ ]:
## Sample response
# Graph showing model for Weight ~ Height
gf_point(Weight ~ Height, data = dat) %>% gf_lm(color = "red")

**Question 4.2 -** Use the `lm` command to find the linear regression model you visualized above. Store the model in an object called `height_model`.

In [ ]:
## Sample response
# Linear model for Weight ~ Height
height_model <- lm(Weight ~ Height, data = dat)
height_model

**Question 4.3 -** Use the `summary` command to find the $R^2$ value of your linear model.

In [ ]:
## Sample response
# r_squared for Weight ~ Height
summary(height_model)

**Question 4.4 -** When attempting to predict the weight of a fish, sceintists will record various measurements. Is this reasonable or is it unecessary? Justify your answers using the $R^2$ values for the `length3_model` and `height_model`.

**Double-click this cell to type your answer here:**

<div class="alert alert-block alert-info"><b>Sample Response:</b> The R^2 value is much higher for the <code>length3_model</code> (85.68%) than the <code>height_model</code> (45.63%). So, full body length is a much stronger predictor of weight than height. When scientists want to use a measurement to predict the weight, it seems that full body weight will be better than height.
</div>